Creates a lazy xarray from a STAC collection and saves the result to a COG file

In [ ]:
#!pip install boto3 rasterio stackstac pystac pystac_client numpy rioxarray python-dotenv

In [1]:
from boto3.session import Session
from rasterio.session import AWSSession
import stackstac
import pystac
import os
import rioxarray
from dotenv import load_dotenv

In [2]:
%load_ext dotenv
%dotenv raw_stacstack_from_cos.credentials
%dotenv raw_stacstack_from_cos.env

aws_access_key_id=os.environ.get('aws_access_key_id')
aws_secret_access_key=os.environ.get('aws_secret_access_key')
cos_endpoint_url=os.environ.get('cos_endpoint_url')
stac_endpoint=os.environ.get('stac_endpoint')
collection_id=os.environ.get('collection_id')

# cog file target location (path/)filename.cog
save_to=os.environ.get('save_to',"cogfile.cog")

# limit number of returned stac items
limit=int(os.environ.get('limit', '2'))

In [3]:
limit = 6 

In [4]:
from pystac_client import Client

client = Client.open(stac_endpoint)

s1_coll = client.get_collection(collection_id)

items = []

In [5]:
item = next(s1_coll.get_items(), None)

i = 0

while item is not None:
    items.append(item.to_dict())
    i = i + 1
    if i >= limit:
        break
    item = next(s1_coll.get_items(), None)

In [6]:
session = Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)

In [7]:
aws_session = AWSSession(
    session=session,
    endpoint_url=cos_endpoint_url,
)

In [8]:
data_array = stackstac.stack(items, epsg=4326, resolution=0.000089831528412, gdal_env=stackstac.DEFAULT_GDAL_ENV.updated(always=dict(session=aws_session)))

/home/romeokienzler/gitco/tensorlakehouse-openeo-driver/venv/lib64/python3.9/site-packages/stackstac/prepare.py:408: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  times = pd.to_datetime(


In [9]:
data_array

<xarray.DataArray 'stackstac-9a95f831385635a107da57821ca362e0' (time: 6,
                                                                band: 1,
                                                                y: 21380,
                                                                x: 57109)> Size: 59GB
dask.array<fetch_raster_window, shape=(6, 1, 21380, 57109), dtype=float64, chunksize=(1, 1, 1024, 1024), chunktype=numpy.ndarray>
Coordinates: (12/17)
  * time             (time) datetime64[ns] 48B 2022-12-12T18:00:23 ... 2022-1...
    id               (time) <U70 2kB 'S1A_IW_GRDH_1SDV_20221212T180023_202212...
  * band             (band) <U4 16B 'data'
  * x                (x) float64 457kB -7.617 -7.617 -7.617 ... -2.487 -2.487
  * y                (y) float64 171kB 60.55 60.55 60.55 ... 58.63 58.63 58.63
    updated          <U25 100B '2022-12-12T18:00:23+00:00'
    ...               ...
    polarization     <U2 8B 'VV'
    tile             <U70 280B 'S1A_IW_GRDH_1SDV_20221212T180023_20221212T180...
    datalayerId      <U2 8B 'DV'
    description      <U73 292B 'Cloud Optimized GeoTIFF (COG) file containing...
    title            <U2 8B 'VV'
    epsg             int64 8B 4326
Attributes:
    spec:        RasterSpec(epsg=4326, bounds=(-7.617444114752364, 58.6301639...
    crs:         epsg:4326
    transform:   | 0.00, 0.00,-7.62|\n| 0.00,-0.00, 60.55|\n| 0.00, 0.00, 1.00|
    resolution:  8.9831528412e-05

In [ ]:
#data_array.isel(time=0).sel(band='data').plot()

In [ ]:
a = data_array.compute()

In [ ]:
a

In [10]:
import time
start = time.time()
data_array.isel(time=0).rio.to_raster(raster_path=save_to, driver="COG")
print(time.time() - start)


RuntimeError: Error reading Window(col_off=13312, row_off=5120, width=1024, height=1024) from 's3://sentinel-1/ds=335/ds=335/dl=50253/dt=20221212/tm=000000/lvl=0/gh=0/S1A_IW_GRDH_1SDV_20221212T180023_20221212T180048_046302_058B97_C1D3_VV.tif': RasterioIOError('Read or write failed. IReadBlock failed at X offset 26, Y offset 42: /vsis3/sentinel-1/ds=335/ds=335/dl=50253/dt=20221212/tm=000000/lvl=0/gh=0/S1A_IW_GRDH_1SDV_20221212T180023_20221212T180048_046302_058B97_C1D3_VV.tif, band 1: IReadBlock failed at X offset 0, Y offset 5429: TIFFReadEncodedStrip() failed.')